In [1]:
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training import Example

In [2]:
nlp=spacy.load('en_core_web_sm')
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [3]:
doc = nlp("Does this model can pickup companies like gucci, jack&jones, tommy hilfiger and c&a?")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

gucci 42 47 PERSON
jack&jones 49 59 ORG
c&a 80 83 GPE


In [7]:
# load the custom_fashion data

with open("custom_fashion_data.txt") as _:
    words = _.readlines()
    print(type(words))


<class 'list'>


In [17]:
words

['["A.P.C.",\n',
 '"A|X Armani Exchange",\n',
 '"A Bathing Ape",\n',
 '"ACME",\n',
 '"Acne Studios",\n',
 '"adidas",\n',
 '"adidas Originals",\n',
 '"Adrianna Papell",\n',
 '"Aerosoles",\n',
 '"AG - Adriano Goldschmied",\n',
 '"agnÃ¨s b.",\n',
 '"Alberta Ferretti",\n',
 '"Alexander McQueen",\n',
 '"Alex Evenings",\n',
 '"Alice & Olivia",\n',
 '"Allied Precision Industries",\n',
 '"Allsaints",\n',
 '"Alternative",\n',
 '"Amazon Basics",\n',
 '"Amazon Collection",\n',
 '"Amazon Essentials",\n',
 '"AMBUSH",\n',
 '"American Apparel",\n',
 '"Americanflat",\n',
 '"Ami",\n',
 '"Amir Slama",\n',
 '"Andrew Marc",\n',
 '"Anne Klein",\n',
 '"Ara",\n',
 '"Ariat",\n',
 '"Armen Living",\n',
 '"ArtistBe",\n',
 '"Artistic Weavers",\n',
 '"Art Maison Canada",\n',
 '"Asos",\n',
 '"Asos Curve",\n',
 '"Asos Petite",\n',
 '"Asos Tall",\n',
 '"Aspesi",\n',
 '"Badgley Mischka",\n',
 '"Baggallini",\n',
 '"Balenciaga",\n',
 '"Bali Intimates",\n',
 '"Bally",\n',
 '"Balmain",\n',
 '"Bandolino",\n',
 '"bareMinera

In [49]:
train_data = []

with open("fashion_brands.txt") as file:
    dataset = file.readlines()
    for sentence in dataset:
        lines = sentence.split("\n")
        non_empty_lines = [line for line in lines if line.strip() != ""]
        print(non_empty_lines)
        sentence = ""
        for line in non_empty_lines:
            sentence += line + "\n"
        print('sentence :' ,sentence)

        sentence = sentence.lower()
        entities = []
        for each_word in words:
            #print(word)
            each_word = each_word.lower().replace('"', "").replace("[", "").replace("]","").replace(",","")
#             print(sentence)
#             print(each_word, word in sentence)
#             print('entering if loop')
            
            if each_word in sentence:
                #print('stage 1')
                #print(f'word is {word}')
                start_index = sentence.index(each_word)
                #print(start_index)
                end_index = len(each_word) + start_index
                #print(end_index)
                # print("word: ", word)
                # print("----------------")
                # print("start index:", start_index)
                # print("end index:", end_index)
                pos = (start_index, end_index, "BRAND")
                #print(pos)
                entities.append(pos)
                #print(entities)
            #print('end of if loop')
        #print('stage 2')
        element = (sentence.rstrip('\n'), {"entities": entities})

        train_data.append(element)
#         print(train_data)
        # print('----------------')
        # print("element:", element)

['I bought my jeans and shoes at adidas']
sentence : I bought my jeans and shoes at adidas

[]
sentence : 
['Bella Hadid in Schiaparelli couture']
sentence : Bella Hadid in Schiaparelli couture

[]
sentence : 
['Meng Li in Balenciaga']
sentence : Meng Li in Balenciaga

[]
sentence : 
['Jodie Turner-Smith in Gucci']
sentence : Jodie Turner-Smith in Gucci

[]
sentence : 
['Lou Doillon in Gucci']
sentence : Lou Doillon in Gucci

[]
sentence : 
['Salma Hayek in Gucci']
sentence : Salma Hayek in Gucci

[]
sentence : 
['Josh Oâ€™Connor in Loewe']
sentence : Josh Oâ€™Connor in Loewe

[]
sentence : 
['Jodie Turner-Smith in Gucci']
sentence : Jodie Turner-Smith in Gucci

[]
sentence : 
['Vanessa Paradis in Chanel']
sentence : Vanessa Paradis in Chanel

[]
sentence : 
['Charlotte Gainsbourg in Saint Laurent']
sentence : Charlotte Gainsbourg in Saint Laurent

[]
sentence : 
['Andie MacDowell in Atelier Versace']
sentence : Andie MacDowell in Atelier Versace

[]
sentence : 
['Diane Kruger in Arman

In [37]:
train_data

[('i bought my jeans and shoes at adidas', {'entities': [(31, 38, 'BRAND')]}),
 ('', {'entities': []}),
 ('bella hadid in schiaparelli couture', {'entities': []}),
 ('', {'entities': []}),
 ('meng li in balenciaga', {'entities': [(11, 22, 'BRAND')]}),
 ('', {'entities': []}),
 ('jodie turner-smith in gucci', {'entities': [(22, 28, 'BRAND')]}),
 ('', {'entities': []}),
 ('lou doillon in gucci', {'entities': [(15, 21, 'BRAND')]}),
 ('', {'entities': []}),
 ('salma hayek in gucci', {'entities': [(15, 21, 'BRAND')]}),
 ('', {'entities': []}),
 ('josh oâ€™connor in loewe', {'entities': [(19, 25, 'BRAND')]}),
 ('', {'entities': []}),
 ('jodie turner-smith in gucci', {'entities': [(22, 28, 'BRAND')]}),
 ('', {'entities': []}),
 ('vanessa paradis in chanel', {'entities': [(19, 26, 'BRAND')]}),
 ('', {'entities': []}),
 ('charlotte gainsbourg in saint laurent', {'entities': [(24, 38, 'BRAND')]}),
 ('', {'entities': []}),
 ('andie macdowell in atelier versace', {'entities': [(27, 35, 'BRAND')]})

In [ ]:
# update the model

# load the needed pipeline
ner = nlp.get_pipe("ner")

# TBD: define the annotations
for _, annotations in train_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# train the model

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# define the number of iterations, the batch size and the drop according to your experience or using an empirical value
# Train model

iter = 200
drop_out = 0.3

with nlp.disable_pipes(*unaffected_pipes):
    for iteration in range(iter):
        print("Iteration #" + str(iteration))

        # Data shuffle for each iteration
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                # Create an Example object
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                # Update the model
                nlp.update([example], losses=losses, drop=drop_out)
        print("Losses:", losses)

# Save the model
output_dir = Path("./ner/")
nlp.to_disk(output_dir)
print("Saved correctly!")

Iteration #0
